# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import func
from collections import OrderedDict

sns.set_style("whitegrid")

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix() + '/src'
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, list_coms

from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, read_image, write_image


In [3]:
def absolute_sum(l):
    la = np.array(l)
    #nabs = np.abs(np.array(la))
    return np.sum(la, axis=0)


## Data Collection

In [4]:
atlas_structures = list_coms('Atlas')
allen_structures = list_coms('Allen')

common_keys = atlas_structures.keys() & allen_structures.keys()
atlas_src = np.array([atlas_structures[s] for s in common_keys])
allen_src = np.array([allen_structures[s] for s in common_keys])
print(atlas_src.shape)

(37, 3)


In [5]:
errors = []
df_list = []
matrix = get_affine_transformation('Atlas')
for structure in common_keys:
    atlas0 = np.array(atlas_structures[structure])
    allen0 = np.array(allen_structures[structure]) 
    transformed = apply_affine_transform(atlas0, matrix)
    transformed = [round(x,2) for x in transformed]
    difference = [round(a - b, 2) for a, b in zip(transformed, allen0)]
    errors.append(difference)
    row = [structure, atlas0, allen0, transformed, difference]
    df_list.append(row)
result = absolute_sum(errors)
print(result)

[-1352.61  1771.02   609.96]


In [6]:
print('Affine transformation matrix')
print(np.array2string(matrix, separator=', '))

Affine transformation matrix
[[ 9.42279538e-01, -1.30121150e-03, -1.50092611e-02,  1.67617121e+02],
 [ 5.07770162e-02,  1.19394497e+00,  8.60586632e-03, -8.14323008e+01],
 [ 4.02430133e-03,  7.16889453e-03,  1.12982209e+00, -6.62364827e+01],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]


In [7]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference']
df = pd.DataFrame(df_list, columns=columns)
columns = ['atlas0', 'allen0', 'transformed', 'difference']
df.index.name = 'Index'
df.sort_values(by=['structure'], inplace=True)
from IPython.display import HTML
HTML(df.to_html(index=False))

structure,atlas0,allen0,transformed,difference
3N_L,"[775.63, 391.54, 557.44]","[910.25, 379.44, 552.1]","[889.6, 430.23, 569.5]","[-20.65, 50.79, 17.4]"
3N_R,"[775.63, 391.54, 582.56]","[910.34, 379.4, 586.44]","[889.22, 430.44, 597.88]","[-21.12, 51.04, 11.44]"
4N_L,"[809.24, 396.73, 545.23]","[958.86, 377.78, 543.25]","[921.45, 438.02, 555.88]","[-37.41, 60.24, 12.63]"
4N_R,"[809.24, 396.73, 594.77]","[958.68, 377.48, 595.29]","[920.7, 438.45, 611.85]","[-37.98, 60.97, 16.56]"
5N_L,"[866.79, 502.13, 432.89]","[1019.3, 528.65, 409.16]","[977.22, 565.82, 429.94]","[-42.08, 37.17, 20.78]"
5N_R,"[866.79, 502.13, 707.11]","[1019.34, 528.55, 729.28]","[973.11, 568.18, 739.76]","[-46.23, 39.63, 10.48]"
6N_L,"[926.23, 512.4, 533.79]","[1077.12, 521.48, 529.09]","[1031.71, 581.97, 544.25]","[-45.41, 60.49, 15.16]"
6N_R,"[926.23, 512.4, 606.21]","[1077.14, 521.46, 609.41]","[1030.62, 582.59, 626.07]","[-46.52, 61.13, 16.66]"
7N_L,"[939.45, 615.73, 455.77]","[1085.27, 677.54, 434.0]","[1045.2, 705.34, 456.9]","[-40.07, 27.8, 22.9]"
7N_R,"[939.45, 615.73, 684.23]","[1085.26, 677.53, 704.52]","[1041.77, 707.31, 715.02]","[-43.49, 29.78, 10.5]"
